In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('all_annotations.txt', sep="\t", header=None, names=["chr","source","type","start","stop","s1","s2","s3","info"])

In [3]:
df.drop(df[df['type'] == "region"].index, inplace = True)

In [4]:
loci = pd.read_csv('top_goodhe', header=None)

In [23]:
new_df=pd.DataFrame()
for index, row in loci.iterrows():
    ch = row[0].split("_")[:2]
    ch = "_".join(ch)
    pos = int(row[0].split("_")[2])
    temp=df[(df['chr']==ch) & (df['start']<=pos) & (df['stop']>=pos)]
    if len(temp) == 0:
        #print("not in annotation")
        new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "not_annot", "gene" : "NA"},  
                        ignore_index = True) 
    if len(temp) == 1:
        print("only one")
        print(temp['type'])
    if len(temp) > 1:
        #print("more than 1")
        #checking for gene overlap
        ids=[]
        for i in range(len(temp)):
            t=temp["info"].iloc[i].split(";")
            res = list(filter(lambda x: "LOC" in x, t))
            if len(res) == 0:
                res = list(filter(lambda x: "GeneID:" in x, t))
                if len(res) != 0:
                    ID = res[0].split("GeneID:",1)[1]
                    ids.append(ID)
            else:
                ID =res[0].split("LOC",1)[1]
                ids.append(ID)
        uni=len(Counter(ids))
        gene_name = ids[0]
        if uni > 1:
            #print("genes overlap")
            new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "genes_overlap", "gene": "NA"},  
                        ignore_index = True) 
        
        if uni == 1:
            if temp['type'].str.contains('gene').any():
                if temp['type'].str.contains('exon').any(): 
                    if temp['type'].str.contains('CDS').any():
                        #print("in exon")
                        new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "exon", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                    else:
                        direction=temp['s2'].iloc[0]
                        e_start=temp[temp["type"]=="exon"].iloc[0].start
                        e_stop=temp[temp["type"]=="exon"].iloc[0].stop
                        g_start=temp[temp["type"]=="gene"].iloc[0].start
                        g_stop=temp[temp["type"]=="gene"].iloc[0].stop
                        if direction == "+":
                            if e_start == g_start:
                                #print('5prime1')
                                new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "5'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                            elif e_stop == g_stop:
                                #print("3prime1")
                                new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "3'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                            else:
                                print("ERROR1") #alternative UTR?
                                #print(temp)
                                new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "alternative UTR", "gene":"NA" },ignore_index = True) 
                        elif direction == "-":
                            if e_start == g_start:
                                #print('3prime2')
                                new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "3'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                            elif e_stop == g_stop:
                                #print("5prime2")
                                new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "5'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True)
                            else:
                                print("ERROR2")
                                new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "alternative UTR", "gene":"NA" },ignore_index = True) 
                        else:
                            print("ERROR3")
                            #print(temp)

                else:
                    #print("in intron")
                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "intron", "gene":f"LOC{ids[0]}" },ignore_index = True) 

            else:
                print("not gene")
    

ERROR1


In [24]:
new_df.pos = new_df.pos.astype(int)

In [64]:
new_df[new_df["hol"]=="exon"]

,chr,gene,hol,pos
5,NW_022145603.1,LOC752046,exon,22321899
47,NW_022145609.1,LOC589355,exon,7625794
48,NW_022145598.1,LOC100891707,exon,14824625
62,NW_022145612.1,LOC105447203,exon,11729117
64,NW_022145596.1,LOC579596,exon,30320957
66,NW_022145604.1,LOC594103,exon,22238462


In [ ]:
#proteins of exons:  homeodomain-interacting protein kinase 2, transcription activator BRG1 (Imschenetzky 2003, might not have role, have to check)

In [ ]:
#locomotion-related protein Hikaru genki, uncharaterised, carbonic anhydrase-related protein (Sp-Cah10L) - melissa's ultimate list, differentially expressed

In [ ]:
#tyrosine-protein phosphatase non-receptor type 4

In [ ]:
#reminder: we want bayenv!

In [65]:
new_df[new_df["hol"]=="3'UTR"]

,chr,gene,hol,pos
6,NW_022145612.1,LOC105446642,3'UTR,21838405
19,NW_022145610.1,LOC752451,3'UTR,27036813
20,NW_022145612.1,LOC581215,3'UTR,51908692
38,NW_022145615.1,LOC581450,3'UTR,488171
40,NW_022145610.1,LOC100890874,3'UTR,30238726
41,NW_022145612.1,LOC105446336,3'UTR,17902099
42,NW_022145615.1,LOC589754,3'UTR,25795628
73,NW_022145612.1,LOC591109,3'UTR,30045837
91,NW_022145612.1,LOC581215,3'UTR,51908700


In [ ]:
#proteins of utrs: uncharacterised, crumbs cell polarity complex component like - in melissas list, "The cleavage to the 16 cell "(Moorhouse 2015)

In [ ]:
# gramd4 death-inducing protein, in supp file of Genome-wide signals of positive selection in strongylocentrotid sea urchins

In [ ]:
# Sp-Brdg1, heparan sulfate 2-O-sulfotransferase 1, atrial natriuretic peptide receptor 1

In [ ]:
# alpha-N-acetylgalactosaminide alpha-2,6-sialyltransferase 2-like,  dedicator of cytokinesis protein 3

In [ ]:
#same as 3rd!!!

In [67]:
new_df[new_df["hol"]=="genes_overlap"]

,chr,gene,hol,pos
43,NW_022145603.1,NA,genes_overlap,5949644
55,NW_022145609.1,NA,genes_overlap,11963040
61,NW_022145597.1,NA,genes_overlap,22428600
72,NW_022145614.1,NA,genes_overlap,15580113


In [ ]:
#receptor for egg jelly 2 protein (rej2) ! interesting, lysosomal alpha-glucosidase pseudogene, 

In [ ]:
#exon of serine/threonine-protein kinase PAK 3 / intron of prostatic spermine-binding protein-like

In [ ]:
#kinesin-like protein KRP180

In [ ]:
#these last 4 I didn't check in melissa's list

In [68]:
new_df[new_df["hol"]=="alternative UTR"]

,chr,gene,hol,pos
53,NW_022145614.1,NA,alternative UTR,8409205


In [ ]:
#RING finger protein 151

In [26]:
new_df.hol.unique()

array(['not_annot', 'exon', "3'UTR", 'intron', 'genes_overlap',
       'alternative UTR'], dtype=object)

## now let's see if there is any in enhancer, outside of intron

are intronic ones located in:
    1 ATAC
    2 lvar
    3 chip 
    4 lnc
    5 promoter
    6 TFBS
    

In [14]:
promoters = pd.read_csv('all_annotations.txt', sep="\t", header=None, names=["chr","source","type","start","stop","s1","s2","s3","info"])
promoters = promoters[promoters['type']=="gene"]

promoters.loc[(promoters["type"]=="gene") & (promoters["s2"]=="+"), "start"] -= 1000
promoters.loc[(promoters["type"]=="gene") & (promoters["s2"]=="-"), "stop"] += 1000

In [31]:
atac = pd.read_csv("atac_mapped_translatedto5.0.csv", header=None, names=["chr","start","stop"])
chip = pd.read_csv('AlxChip_mapped_translatedto5.0.txt', sep="\t", header=None, names=["chr","start","stop"])
lvar = pd.read_csv('lvar_spur_48_50_mapped_translatedto50.csv', header=None, names=["chr","start","stop"])
lnc = pd.read_csv("lncRNA_mapped_translatedto5.0.csv", header=1, names=["chr","start","stop"])

In [27]:
mis = new_df[new_df['hol']=="not_annot"]

In [57]:
enh_check=pd.DataFrame()
for index, row in mis.iterrows():
    ch = row.chr
    pos = row.pos
    temp=atac[(atac['chr']==ch) & (atac['start']<=pos) & (atac['stop']>=pos)]
    if len(temp) > 0:
        print("in atac")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "atac" },ignore_index = True) 
    temp=chip[(chip['chr']==ch) & (chip['start']<=pos) & (chip['stop']>=pos)]
    if len(temp) > 0:
        print("in chip")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "chip" },ignore_index = True)
    temp=lvar[(lvar['chr']==ch) & (lvar['start']<=pos) & (lvar['stop']>=pos)]
    if len(temp) > 0:
        print("in lvar")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lvar" },ignore_index = True)
    temp=lnc[(lnc['chr']==ch) & (lnc['start']<=pos) & (lnc['stop']>=pos)]
    if len(temp) > 0:
        print("in lnc")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lnc" },ignore_index = True)
    temp=promoters[(promoters['chr']==ch) & (promoters['start']<=pos) & (promoters['stop']>=pos)]
    if len(temp) > 0:
        print("in promoters")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "promoter" },ignore_index = True)

0
1
2
3
4
7
10
11
12
14
15
in promoters
16
17
18
21
in promoters
22
23
24
25
28
29
31
33
in lnc
37
39
44
49
in promoters
54
56
57
58
69
70
71
74
75
76
77
78
79
80
82
85
86
87
88
90


In [58]:
enh_check

,chr,hol,pos
0,NW_022145598.1,promoter,19923027.0
1,NW_022145612.1,promoter,51906905.0
2,NW_022145603.1,lnc,30428006.0
3,NW_022145600.1,promoter,16886774.0


In [ ]:
#promoters: LOC115923223, LOC581330, LOC589501, nothing very interesting

## now let's see if there is any in enhancer, inside of intron

In [59]:
mis = new_df[new_df['hol']=="intron"]

In [62]:
int_check=pd.DataFrame()
for index, row in mis.iterrows():
    ch = row.chr
    pos = row.pos
    temp=atac[(atac['chr']==ch) & (atac['start']<=pos) & (atac['stop']>=pos)]
    if len(temp) > 0:
        print("in atac")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "atac" },ignore_index = True) 
    temp=chip[(chip['chr']==ch) & (chip['start']<=pos) & (chip['stop']>=pos)]
    if len(temp) > 0:
        print("in chip")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "chip" },ignore_index = True)
    temp=lvar[(lvar['chr']==ch) & (lvar['start']<=pos) & (lvar['stop']>=pos)]
    if len(temp) > 0:
        print("in lvar")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lvar" },ignore_index = True)
    temp=lnc[(lnc['chr']==ch) & (lnc['start']<=pos) & (lnc['stop']>=pos)]
    if len(temp) > 0:
        print("in lnc")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lnc" },ignore_index = True)

in lnc
in lnc


In [63]:
int_check

,chr,hol,pos
0,NW_022145603.1,lnc,15709285.0
1,NW_022145601.1,lnc,22606114.0


In [ ]:
#there is no point to check TFBS if no putative enhancer is recovered